In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import tensorflow_datasets as tfds
import tensorflow as tf
from transformer import Transformer
from scheduler import TransformerScheduler

C:\Users\Han SeongMin\AppData\Local\Temp\ipykernel_10992\3242935675.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


[손실 함수 정의]
예제는 다중 클래스 분류 문제. 이때 레이블이 정수 형태이므로 손실 함수는 SparseCategoricalCrossentropy 사용

In [2]:
def loss_function(ans, pred):
    """
    다중 클래스 분류 문제를 위한 손실 함수 정의
    
    :param ans: 해당 데이터의 실제 정답
    :param pred: 모델이 생성해낸 예측 레이블
    :return: 손실값
    """
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')(ans, pred)
    mask = tf.cast(tf.not_equal(ans, 0), tf.float32)
    loss = tf.multiply(loss, mask)
    
    return tf.reduce_mean(loss)

[데이터 로드]
챗봇 데이터를 로드
학습 기반 토크나이저 사용을 위해 구두점 처리

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('CHatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [4]:
print(f'샘플의 개수 : {len(train_data)}')

샘플의 개수 : 11823


In [5]:
print(train_data.isnull().sum())

Q        0
A        0
label    0
dtype: int64


In [6]:
# 구두점 제거 대신 띄어쓰기를 추가하여 다른 문자와 구분
# 정규식 사용하여 처리
questions = []
for sentence in train_data['Q']:
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    questions.append(sentence)
    
answers = []
for sentence in train_data['A']:
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    answers.append(sentence)

In [7]:
print(questions[:5])
print(answers[:5])

['12시 땡 !', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']
['하루가 또 가네요 .', '위로해 드립니다 .', '여행은 언제나 좋죠 .', '여행은 언제나 좋죠 .', '눈살이 찌푸려지죠 .']


[단어 집합 생성]
서브워드 텍스트 인코더를 사용하여 서브워드로 구성된 단어 집합 생성

In [8]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)

In [9]:
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
VOCAB_SIZE = tokenizer.vocab_size + 2

In [10]:
print(f'START_TOKEN : {START_TOKEN}')
print(f'END_TOKEN : {END_TOKEN}')
print(f'VOCAB_SIZE : {VOCAB_SIZE}')

START_TOKEN : [8178]
END_TOKEN : [8179]
VOCAB_SIZE : 8180


[정수 인코딩과 패딩]
토크나이저의 .encode()를 사용하여 정수 인코딩

In [11]:
sample_string = questions[20]
tokenized_string = tokenizer.encode(sample_string)
print(f'원본 문장 : {sample_string}')
print(f'encode 후 : {tokenized_string}')
print(f'decode 후 : {tokenizer.decode(tokenized_string)}')

원본 문장 : 가스비 비싼데 감기 걸리겠어
encode 후 : [5766, 611, 3509, 141, 685, 3747, 849]
decode 후 : 가스비 비싼데 감기 걸리겠어


In [12]:
for token in tokenized_string:
    print(f'{token} ----> {tokenizer.decode([token])}')

5766 ----> 가스
611 ----> 비 
3509 ----> 비싼
141 ----> 데 
685 ----> 감기 
3747 ----> 걸리
849 ----> 겠어


In [13]:
MAX_LENGTH = 40

def encode_and_padding(inputs, outputs):
    """
    1. 토크나이저로 인코딩
    2. START_TOKEN, END_TOKEN 추가
    3. 패딩 수행
    
    :param inputs: 데이터 셋의 입력
    :param outputs: 데이터 셋의 출력
    :return: 인코딩된 입력과 출력 리스트
    """
    encoded_inputs, encoded_outputs = [], []
    
    for input_sentence, output_sentence in zip(inputs, outputs):
        encoded_inputs.append(START_TOKEN + tokenizer.encode(input_sentence) + END_TOKEN)
        encoded_outputs.append(START_TOKEN + tokenizer.encode(output_sentence) + END_TOKEN)
        
    encoded_inputs = tf.keras.preprocessing.sequence.pad_sequences(encoded_inputs, maxlen=MAX_LENGTH, padding='post')
    encoded_outputs = tf.keras.preprocessing.sequence.pad_sequences(encoded_outputs, maxlen=MAX_LENGTH, padding='post')
    
    return encoded_inputs, encoded_outputs

In [14]:
encoded_questions, encoded_answers = encode_and_padding(questions, answers)

print(f'질문 데이터의 크기 : {encoded_questions.shape}')
print(f'답변 데이터의 크기 : {encoded_answers.shape}')

print(f'0번 샘플 질문 데이터 : {encoded_questions[0]}')
print(f'0번 샘플 답변 데이터 : {encoded_answers[0]}')

질문 데이터의 크기 : (11823, 40)
답변 데이터의 크기 : (11823, 40)
0번 샘플 질문 데이터 : [8178 7915 4207 3060   41 8179    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
0번 샘플 답변 데이터 : [8178 3844   74 7894    1 8179    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]


[인코더와 디코더의 입력 및 레이블 만들기]
tf.data.Dataset을 사용하여 데이터를 배치 단위로 불러올 수 있다.

In [15]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices(({
    'inputs': encoded_questions,
    'dec_inputs': encoded_answers[:, :-1]
},
{
    'outputs': encoded_answers[:, 1:]
}))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [16]:
for i in dataset:
    print(i)
    break

({'inputs': <tf.Tensor: shape=(64, 40), dtype=int32, numpy=
array([[8178,   51,  371, ...,    0,    0,    0],
       [8178, 4099,  354, ...,    0,    0,    0],
       [8178,  618,  258, ...,    0,    0,    0],
       ...,
       [8178, 1096,  350, ...,    0,    0,    0],
       [8178,  882,  220, ...,    0,    0,    0],
       [8178,  770,  782, ...,    0,    0,    0]])>, 'dec_inputs': <tf.Tensor: shape=(64, 39), dtype=int32, numpy=
array([[8178,   51,  371, ...,    0,    0,    0],
       [8178,  656, 5311, ...,    0,    0,    0],
       [8178, 7457,    1, ...,    0,    0,    0],
       ...,
       [8178, 1655, 5317, ...,    0,    0,    0],
       [8178, 4509,   25, ...,    0,    0,    0],
       [8178, 7690,  906, ...,    0,    0,    0]])>}, {'outputs': <tf.Tensor: shape=(64, 39), dtype=int32, numpy=
array([[  51,  371,  174, ...,    0,    0,    0],
       [ 656, 5311,    5, ...,    0,    0,    0],
       [7457,    1, 8179, ...,    0,    0,    0],
       ...,
       [1655, 5317, 2528,

[트랜스포머 만들기]
인풋 모양은 (2(인코더 입력, 디코더 입력), batch_size, MAX_LENGTH)을 의미

In [17]:
D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1

transformer = Transformer(vocab_size=VOCAB_SIZE,
                          d_model=D_MODEL,
                          num_layers=NUM_LAYERS,
                          num_heads=NUM_HEADS,
                          d_ff=DFF,
                          dropout=DROPOUT)

transformer.build(input_shape=(2, BATCH_SIZE, MAX_LENGTH))

ValueError: You cannot build your model by calling `build` if your layers do not support float type inputs. Instead, in order to instantiate and build your model, call your model on real tensor data (of the correct dtype).

The actual error from `call` is: 'SymbolicTensor' object cannot be interpreted as an integer.

In [ ]:
learning_rate = TransformerScheduler(d_model=D_MODEL)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, (-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

transformer.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [ ]:
EPOCHS = 50
transformer.fit(dataset, epochs=EPOCHS)